# Using Multi-Agents to create a Blob Post for GPT-4 Aniversary

| Date | Action | Description |
|-|-|-|
| 2024-03-15 | Original | Original code |
| 2024-03-16 | Updated  | Improve prompts, printing and add the final blog post |


## Introduction
In this blog post, we will use multi-agent system to create a blog post for GPT-4 Aniversary.



## Step01: Import libs and Variables


In [30]:
import os
from dotenv import load_dotenv
from langchain_community.utilities import SerpAPIWrapper
import functools, operator
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_core.messages import BaseMessage, HumanMessage
from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, END
from langchain.tools import tool
from langchain_openai import ChatOpenAI
from typing import Annotated, Sequence, TypedDict

from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    ChatMessage,
    FunctionMessage,
    HumanMessage,
    SystemMessage
)

# Load environment variables
load_dotenv()

True

## Step 02: Let's initialize and test our objects and tools

In [31]:
# Let's test the SerpAPIWrapper
search = SerpAPIWrapper()
search.run("What is the capital of Chile?")

'Santiago'

In [32]:
# Create the LLM Object
llm = ChatOpenAI(model="gpt-4-turbo-preview")
llm.invoke('Hello!')

AIMessage(content='Hello! How can I assist you today?', response_metadata={'finish_reason': 'stop', 'logprobs': None})

## Step 03: Let's create the multi-agent system

### First we will create some tools

In [33]:
@tool("web_search")
def web_search(query: str) -> str:
    """Search with Google SERP API by a query"""
    search = SerpAPIWrapper()
    return search.run(query)

In [34]:
class AgentState(TypedDict):
    # The annotation tells the graph that new messages will always
    # be added to the current states
    messages: Annotated[Sequence[BaseMessage], operator.add]
    # The 'next' field indicates where to route to next
    next: str

In [35]:
# Definer an agent creator and nod
def create_agent(llm: ChatOpenAI, tools: list, system_prompt: str):
    # Each worker node will be given a name and some tools.
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                system_prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent = create_openai_tools_agent(llm, tools, prompt)
    executor = AgentExecutor(agent=agent, tools=tools)
    return executor

def agent_node(state, agent, name):
    result = agent.invoke(state)
    return {"messages": [HumanMessage(content=result["output"], name=name)]}

In [36]:
members = ["Researcher", "Writer", "Critic"]
system_prompt = (
    "As the Supervisor at an editorial office, your role is to manage a conversation among the "
    "following workers: {members}. Given the user's request below, determine which worker should "
    "act next and provide them with clear instructions and guidance."
    "\n\nThe typical workflow in this agency follows these steps:"
    "\n1. The Researcher:"
    "\n   - Conducts thorough research on the topic using reliable sources."
    "\n   - Gathers relevant information, data, and insights to support the content."
    "\n   - Provides a summary of the key findings and recommendations for the Writer."
    "\n2. The Writer:"
    "\n   - Reviews the research provided by the Researcher."
    "\n   - Creates a well-structured and engaging first draft of the blog post."
    "\n   - Incorporates examples, statistics, and expert quotes to support the arguments."
    "\n   - Optimizes the content for search engines and formats it for readability."
    "\n3. The Critic:"
    "\n   - Evaluates the first draft created by the Writer."
    "\n   - Provides constructive feedback and suggestions for improvement."
    "\n   - Focuses on areas such as structure, clarity, tone, grammar, and effectiveness."
    "\n4. The Writer:"
    "\n   - Incorporates the feedback provided by the Critic."
    "\n   - Refines and polishes the blog post to create a final, high-quality piece."
    "\n   - Ensures that the content meets the requirements and expectations set by the user."
    "\n\nAs the Supervisor, your task is to:"
    "\n- Assign tasks to the appropriate worker based on the current stage of the workflow."
    "\n- Provide clear instructions and guidance to ensure a smooth collaboration among the workers."
    "\n- Monitor the progress of the task and address any issues or concerns that may arise."
    "\n- Review the final document to ensure it meets the required standards before marking the task as complete."
    "\n\nEach worker will perform their assigned task and respond with the results and their current status. "
    "Continue this process until the task is complete, then respond with 'FINISH'."
)

options = ["FINISH"] + members
# Using openai function calling can make output parsing easier for us
function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {
            "next": {
                "title": "Next",
                "anyOf": [
                    {"enum": options},
                ],
            }
        },
        "required": ["next"],
    },
}
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "Given the conversation above, who should act next?"
            " Or should we FINISH? Select one of: {options}",
        ),
    ]
).partial(options=str(options), members=", ".join(members))

supervisor_chain = (
    prompt
    | llm.bind_functions(functions=[function_def], function_call="route")
    | JsonOutputFunctionsParser()
)

In [37]:
# Define the Critic agent
critic_profile = """
As a Critic, you are an AI agent tasked with providing constructive feedback and suggestions to enhance the quality of written content. Your primary goal is to assess the strengths and weaknesses of the given text and offer specific, actionable advice for improvement. Focus on the following key areas:

1. Structure and Organization:
   - Evaluate the content's organization and logical flow.
   - Suggest improvements to the structure, such as rearranging paragraphs or adding subheadings, to enhance readability and clarity.

2. Clarity and Coherence:
   - Assess the clarity and coherence of the ideas presented.
   - Identify ambiguous or confusing passages and provide suggestions for improving clarity, such as rephrasing sentences or adding explanations.

3. Tone and Style:
   - Analyze the tone and style of the writing and determine its appropriateness for the target audience and purpose.
   - Offer suggestions for adjusting the tone or style to better engage and resonate with the intended readers.

4. Grammar and Syntax:
   - Check for grammatical errors, punctuation mistakes, or syntactical issues.
   - Provide specific corrections and explanations to help the writer improve their language skills.

5. Effectiveness and Impact:
   - Evaluate the overall effectiveness and impact of the content in achieving its intended purpose.
   - Offer insights on how to strengthen arguments, provide more compelling evidence, or enhance the persuasive power of the writing.

When providing feedback, maintain a constructive and supportive tone. Acknowledge the strengths of the writing while offering specific, actionable suggestions for improvement. Your goal is to help the writer refine their work and create high-quality, impactful content.
"""

critic_agent = create_agent(
    llm=llm,
    tools=[web_search],
    system_prompt=critic_profile
)

In [38]:
# Define the research agent
researcher_profile = """
As the Researcher, you are an AI agent responsible for finding relevant, high-quality information using search engines and other resources. Your primary goal is to gather and synthesize information to provide the Writer with the necessary insights, data, and context to create a well-informed and engaging blog post. To achieve this, follow these steps:

1. Understand the research question or topic:
   - Analyze the given research question or topic to identify the key concepts, themes, and objectives.
   - Break down the topic into smaller, more manageable subtopics or aspects to guide your research.

2. Employ advanced search techniques:
   - Use Boolean operators (AND, OR, NOT) to combine search terms and refine results.
   - Utilize phrase searches to find exact matches and domain filtering to focus on relevant websites or sources.
   - Adapt your search strategies based on the initial results to optimize the relevance and quality of the information found.

3. Evaluate the credibility and relevance of sources:
   - Assess the credibility of each source by considering factors such as the author's expertise, the publication's reputation, and the date of publication.
   - Prioritize sources that are most relevant to the research question, providing insights, data, or context that will enhance the Writer's understanding of the topic.

4. Synthesize and summarize the findings:
   - Extract the key information, insights, and data from the selected sources.
   - Synthesize the findings into concise summaries, highlighting the most important points and their relevance to the research question.
   - Organize the information in a logical manner, making it easy for the Writer to understand and incorporate into the blog post.

5. Provide additional context and recommendations:
   - Offer background information, definitions, or explanations of complex concepts to ensure the Writer has a comprehensive understanding of the topic.
   - Suggest potential angles, perspectives, or examples that the Writer can explore in the blog post to make it more engaging and informative.
   - Recommend additional resources, such as related articles, studies, or expert opinions, that can further enhance the quality and depth of the content.

Throughout the research process, maintain a focus on finding the most relevant, reliable, and up-to-date information. Be prepared to iterate on your search strategies and refine your findings as needed to ensure the Writer has access to the best possible resources for creating a high-quality blog post.
"""

researcher_engine_agent = create_agent(
    llm=llm,
    tools=[web_search],
    system_prompt=researcher_profile
)

In [39]:
# Define the research agent
blog_writer_profile = """
As the Writer, you are an expert in creating engaging, informative, and well-structured blog content on a wide range of topics. Your primary goal is to write a comprehensive blog post based on the given topic, target audience, and any additional requirements provided. Follow these steps to ensure a high-quality outcome:

1. Research and Planning:
   - Conduct thorough research using reliable sources to gather relevant information, data, and insights to support your content.
   - Analyze the target audience to understand their needs, preferences, and level of knowledge on the topic.
   - Identify the key points, arguments, and takeaways you want to convey in the blog post.

2. Blog Post Structure:
   - Craft a compelling headline that accurately reflects the content and captures the reader's attention.
   - Write an engaging introduction that hooks the reader and provides a clear overview of the main points to be discussed.
   - Organize the body of the blog post into logical sections and subsections, each focusing on a specific aspect of the topic.
   - Use examples, statistics, and expert quotes to support your arguments and provide value to the reader.
   - Conclude the blog post with a summary of the key takeaways and a call-to-action that encourages the reader to engage further with the topic or take a specific action.

3. Writing Style and Tone:
   - Employ a clear, concise, and engaging writing style that resonates with the target audience.
   - Adapt your tone and language to match the specific requirements of the topic and the preferences of the target audience.
   - Use appropriate transitions and linking words to ensure a smooth flow of ideas throughout the blog post.
   - Incorporate storytelling techniques, analogies, or real-life examples to make the content more relatable and memorable.

4. Search Engine Optimization (SEO):
   - Research and include relevant keywords and phrases naturally throughout the blog post to improve its visibility in search engine results.
   - Optimize the blog post's meta tags, including the title tag and meta description, to attract clicks from search engine results pages.
   - Use appropriate header tags (H1, H2, H3) to structure the content and signal the importance of different sections to search engines.

5. Formatting and Visual Appeal:
   - Break up the content into shorter paragraphs and use bullet points or numbered lists to enhance readability.
   - Incorporate relevant images, videos, or infographics to support the content and provide visual interest.
   - Use appropriate formatting techniques, such as bold or italic text, to highlight key points or emphasize important information.
   - Ensure that the blog post is mobile-friendly and displays correctly across different devices and screen sizes.

Before finalizing the blog post, review and revise your work to ensure that it is coherent, error-free, and effectively communicates your intended message. Your ultimate goal is to create a blog post that informs, engages, and inspires readers to take action or explore the topic further.
"""

writer_operator_agent = create_agent(
    llm=llm,
    tools=[web_search],
    system_prompt=blog_writer_profile
)

In [40]:
# Add the agents to the graph.
researcher_engine_node = functools.partial(agent_node, agent=researcher_engine_agent, name="Researcher")
writer_operator_node = functools.partial(agent_node, agent=writer_operator_agent, name="Writer")
critic_operator_node = functools.partial(agent_node, agent=critic_agent, name="Critic")

In [41]:
# Define the state
workflow = StateGraph(AgentState)
workflow.add_node("Researcher", researcher_engine_node)
workflow.add_node("Writer", writer_operator_node)
workflow.add_node("Critic", critic_operator_node)
workflow.add_node("Supervisor", supervisor_chain)

In [42]:
for member in members:
    workflow.add_edge(member, "Supervisor")

conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("Supervisor", lambda x: x["next"], conditional_map)

workflow.set_entry_point("Supervisor")

graph = workflow.compile()

## Step 04: Execute the Agent

In [43]:
required_task = (
    "Task: Write a compelling blog article celebrating the anniversary of GPT-4 and its impact on the world."
    "\n\nRequirements:"
    "\n1. Length: The article should be at least 1,000 words long to provide a comprehensive exploration of the topic."
    "\n2. Key Points:"
    "\n   - Discuss how GPT-4 is changing various aspects of life, such as communication, education, and business."
    "\n   - Highlight the potential benefits and opportunities created by GPT-4 and similar AI technologies."
    "\n   - Address the risks and challenges associated with the development and deployment of advanced AI systems."
    "\n   - Explore the future implications of GPT-4 and how it may shape the world in the coming years."
    "\n3. Tone and Style:"
    "\n   - Create an engaging and thought-provoking piece that raises questions and encourages readers to consider the future of AI."
    "\n   - Use a clear, concise, and compelling writing style that resonates with a wide audience."
    "\n   - Incorporate real-life examples, anecdotes, or expert opinions to support your arguments and make the content more relatable."
    "\n   - Ensure that the blog post reads as if it were written by a human, with a natural flow and appropriate emotional touch."
    "\n4. Structure:"
    "\n   - Begin with a captivating introduction that hooks the reader and sets the context for the article."
    "\n   - Organize the main points into logical sections and subsections, each focusing on a specific aspect of GPT-4's impact."
    "\n   - Use appropriate transitions and linking words to ensure a smooth flow of ideas throughout the post."
    "\n   - Conclude with a powerful message that summarizes the key takeaways and leaves the reader with food for thought."
    "\n5. Additional Requirements:"
    "\n   - Conduct thorough research using reliable sources to gather relevant information, statistics, and expert opinions."
    "\n   - Optimize the blog post for search engines by including relevant keywords and meta tags."
    "\n   - Format the post using appropriate headings, subheadings, bullet points, and images to enhance readability and visual appeal."
    "\n   - Proofread and edit the final draft to ensure it is error-free and polished."
)

def print_step(step, message):
    print(f"\n\n{step} \n{message}\n")

steps = []
for s in graph.stream(
    {
        "messages": [
            HumanMessage(content=required_task)
        ]
    }
):
    steps.append(s)
    if "__end__" not in s:
        if "Supervisor" in s:
            print_step("---------- Supervisor ----------", f"Assigned the task to the {s['Supervisor']['next']}.")
        elif "Researcher" in s:
            print_step("---------- Researcher ----------", s['Researcher']['messages'][0].content)
        elif "Writer" in s:
            print_step("---------- Writer ----------", s['Writer']['messages'][0].content)
        elif "Critic" in s:
            print_step("---------- Critic ----------", s['Critic']['messages'][0].content)



---------- Supervisor ----------: 
Assigned the task to the Researcher.



---------- Researcher ----------: 
To begin crafting a compelling article celebrating the anniversary of GPT-4 and its monumental impact on the world, we'll outline key points based on the research gathered across various domains such as communication, education, business, the benefits and challenges of GPT-4, and its future implications.

---

**Introduction:**

In the digital era, where innovation is the cornerstone of progress, the advent of GPT-4 has marked a significant milestone in the journey of artificial intelligence. As we celebrate its anniversary, it's crucial to delve into how this groundbreaking technology is reshaping the landscapes of communication, education, business, and beyond. This article explores the multifaceted impacts of GPT-4, highlighting the potential benefits and challenges it brings along, and speculating on its future implications.

**Impact on Communication:**

GPT-4's sophisti

In [54]:
print(steps[-1]['__end__']['messages'][-1].content)

Thank you for the detailed feedback. Taking your suggestions into account, let's refine the article to further enhance its quality, clarity, and impact. Here's the revised version:

---

## A Year of GPT-4: Revolutionizing Our Daily Lives in Unforeseen Ways

Imagine waking up to a world where your digital assistant not only understands your every word but anticipates your needs with empathy and accuracy. This isn't a scene from a sci-fi movie—it's the reality brought closer by GPT-4. As we commemorate its first anniversary, it's time to explore how this pioneering technology is reshaping our means of communication, transforming education, revolutionizing business practices, and redefining our societal norms.

### Bridging Communication Gaps with AI Empathy

GPT-4 has ushered in a new era of communication, enabling AI to interact with the depth and subtlety of human conversation. Through advancements such as improved natural language processing and understanding multimodal inputs, it's 